<a href="https://colab.research.google.com/github/Pradyumnan-Ramanathan/demo-repo/blob/main/food_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import files
import zipfile
#importing the file
#file structure : desi
#                  ---->training --->biryani
#                                ---->butter_chicken
#                  ---->test -------->biryani
#                            -------->butter_chicken
print("Upload your dataset (e.g., a .zip file)...")
uploaded = files.upload()

Upload your dataset (e.g., a .zip file)...


Saving desi (2).zip to desi (2).zip


In [ ]:
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('food')  # Extracts into a folder named 'food'

Extracting desi (2).zip...


**Verifying the structure of training and test set**

In [ ]:
train_dir = '/content/food/desi/training'
test_dir = '/content/food/desi/test'

print(f"Training folder contents: {os.listdir(train_dir)}")
print(f"Test folder contents: {os.listdir(test_dir)}")


Training folder contents: ['biryani', 'butter_chicken']
Test folder contents: ['biryani', 'butter_chicken']


**Data preprocessing**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, #normalise the pixels
    rotation_range=20,# rotate the image by an angle
    width_shift_range=0.2, #horizontal shift
    height_shift_range=0.2, #vertical shift
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True # flipping the image
)

test_datagen = ImageDataGenerator(rescale=1./255)
#training set
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),# commonly used size by pre trained models
    batch_size=32,# number of images per batch
    class_mode='binary'
)
# test set
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


**Building the convolutional neural network**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),#first layer, activation is commonly used for computation
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),#2nd layer
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),#3rd layer
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),# to prevent overfitting
    Dense(1, activation='sigmoid')  #commonly used for the final layer,thredhold may also be considered
])


**Compiling the model**

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


**Training the model**

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.6281 - loss: 0.6811 - val_accuracy: 0.6500 - val_loss: 0.5891
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.7414 - loss: 0.5762 - val_accuracy: 0.6000 - val_loss: 0.5970
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.6677 - loss: 0.5627 - val_accuracy: 0.7500 - val_loss: 0.5702
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.7906 - loss: 0.5202 - val_accuracy: 0.8000 - val_loss: 0.5517
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.8109 - loss: 0.4752 - val_accuracy: 0.6500 - val_loss: 0.4842
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.8010 - loss: 0.4711 - val_accuracy: 0.7000 - val_loss: 0.4709
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7505 - loss: 0.4890 - val_accuracy: 0.8000 - val_loss: 0.4398
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.8375 - loss: 0.5282 - val_accuracy: 0.8000 - val_loss: 0.4586
Epoch 9/

**Evaluating the model performance on test data**

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step - accuracy: 0.8500 - loss: 0.3986
Test Accuracy: 85.00%


**Uploading a random image to test the model**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving biryani.jpeg to biryani.jpeg


**Preprocessing the new image**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array


image_path = "/content/biryani.jpeg" #new image path
image_size = (224, 224)
img = load_img(image_path, target_size=image_size)
img_array = img_to_array(img)
img_array = img_array / 255.0  #normalising the image
img_array = np.expand_dims(img_array, axis=0)

**Predicting the new image**

In [ ]:
predictions = model.predict(img_array)# predicting the new image


class_labels = ["biryani", "butter_chicken"]


predicted_index = np.argmax(predictions[0])
predicted_label = class_labels[predicted_index]
predicted_probability = predictions[0][predicted_index]


threshold = 0.5
if predicted_probability >= threshold:
    print(f"The given image is classified as: {predicted_label} with confidence {predicted_probability:.2f}")
else:
    print(f"The given image does not match 'biryani' or 'butter chicken'. It is classified as: Neither.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
The given image is classified as: biryani with confidence 0.68


In [ ]:
model.save('/content/my_model.h5')  # Save as a single HDF5 file
